In [43]:
import pandas as pd
pd.set_option('display.max_rows' , 200)

In [44]:
company_name = 'TCS'

In [45]:
share_price_in = 1000000;

In [46]:
def calculate_growth(initial , final) :
    if not initial :
        return 1
    return round(((final - initial) / initial) * 100,2)

In [47]:
def get_share_price_in(price) :
    return round(price / share_price_in,2)

    

In [48]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [49]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [50]:
income_statement.rename(columns={company_name+'_income-statement_Annual_As_Originally_Reported' : 'items'} , inplace=True)
balance_sheet.rename(columns={company_name+'_balance-sheet_Annual_As_Originally_Reported' : 'items'} , inplace=True)
cashflow_statement.rename(columns={company_name+'_cash-flow_Annual_As_Originally_Reported' : 'items'} , inplace=True)

In [51]:
basic_eps_filter = income_statement['items'] == 'Basic EPS'
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [52]:
WASO_filter = (income_statement['items'] == 'Basic Weighted Average Shares Outstanding') | (income_statement['items'] == 'Diluted Weighted Average Shares Outstanding') | (income_statement['items'] == 'Basic WASO') | (income_statement['items'] == 'Diluted WASO')
waso_pos = income_statement[WASO_filter].index

In [53]:
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [54]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [55]:
temp_df = temp_df.apply(get_share_price_in)

In [56]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [57]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [58]:
WASO_filter = (income_statement['items'] == 'Basic Weighted Average Shares Outstanding') | (income_statement['items'] == 'Diluted Weighted Average Shares Outstanding') | (income_statement['items'] == 'Basic WASO') | (income_statement['items'] == 'Diluted WASO')

In [59]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [60]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [61]:
cashflow_statement

,items,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,"Cash Flow from Operating Activities, Indirect",116149.6,147514.1,193687.8,191154.4,252230.0,250670.0,285930.0,323690.0,388020.0,399490.0,410970.0
1,Net Cash Flow from Continuing Operating Ac...,116149.6,147514.1,193687.8,191154.4,252230.0,250670.0,285930.0,323690.0,388020.0,399490.0,410970.0
2,Cash Generated from Operating Activities,164367.7,217950.4,268505.4,266843.2,331690.0,326760.0,385510.0,382150.0,478940.0,514350.0,534200.0
3,Income/Loss before Non-Cash Adjust...,180897.3,254018.6,262984.9,316758.7,263570.0,258800.0,315620.0,324470.0,325620.0,384490.0,397490.0
4,Total Adjustments for Non-Cash Items,1135.8,-1560.3,-1951.7,-1679.7,74430.0,70050.0,91580.0,104350.0,144690.0,157550.0,162870.0
5,"Depreciation, Amortization and...",10799.2,13491.5,13089.4,19479.6,19870.0,20140.0,20560.0,35290.0,40650.0,46040.0,48800.0
6,Depreciation and Amortizat...,10799.2,13491.5,13089.4,19479.6,19870.0,20140.0,20560.0,35290.0,40650.0,46040.0,48800.0
7,"Deferred Taxes, Non-Cash Adjus...",0.0,0.0,0.0,0.0,81560.0,82120.0,100010.0,98010.0,111980.0,132380.0,136780.0
8,Interest and Finance Income/Ex...,-9912.5,-13014.8,-14924.2,-16957.0,-22310.0,-23930.0,-25640.0,-26380.0,-18670.0,-18790.0,-19650.0
9,"Other Provisions, Non-Cash Adj...",105.3,1011.3,1650.2,1218.1,0.0,0.0,0.0,0.0,12180.0,0.0,0.0


In [62]:
income_statement

,items,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Gross Profit,362970.20,488606.70,521114.30,622633.10,535370.00,540080.00,659470.00,690920.00,773020.00,914870.00,1016280.00
1,Total Revenue,629894.80,818093.60,946484.10,1086462.10,1179660.00,1231040.00,1464630.00,1569490.00,1641770.00,1917540.00,2168870.00
2,Business Revenue,629903.00,818093.60,946484.10,1086462.10,1179660.00,1231040.00,1464630.00,1569490.00,1641770.00,1917540.00,2168870.00
3,Excise Taxes,-8.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Cost of Revenue,-266924.60,-329486.90,-425369.80,-463829.00,-644290.00,-690960.00,-805160.00,-878570.00,-868750.00,-1002670.00,-1152590.00
5,Cost of Goods and Services,-26525.00,-30886.80,-38358.30,-46138.20,-28080.00,-27000.00,-22700.00,-19050.00,-14620.00,-11630.00,-15210.00
6,"Staff Cost, Cost of Revenue",-240399.60,-298600.10,-387011.50,-417690.80,-616210.00,-663960.00,-782460.00,-859520.00,-854130.00,-991040.00,-1137380.00
7,Operating Income/Expenses,-192894.20,-250606.10,-294143.00,-336086.20,-230880.00,-233000.00,-284970.00,-305120.00,-346200.00,-428990.00,-492340.00
8,"Selling, General and Administrative Expenses",-119905.90,-155903.40,-176887.10,-193945.90,-99210.00,-100540.00,-126510.00,-145290.00,-215110.00,-279090.00,0.00
9,Staff Costs,-78819.00,-106262.90,-122232.00,-136786.50,0.00,0.00,0.00,0.00,-64010.00,-84500.00,0.00


In [63]:
balance_sheet

,items,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Total Assets,522672.2,671377.8,736608.8,893843.8,1032520.0,1062960.0,1149430.0,1208990.0,1307590.0,1415140.0
1,Total Current Assets,315765.6,428976.9,488130.0,630673.9,805260.0,812240.0,921310.0,902370.0,992800.0,1083100.0
2,"Cash, Cash Equivalents and Short Term ...",76818.6,155747.2,197359.7,287039.5,457030.0,426810.0,419390.0,357250.0,383830.0,484330.0
3,Cash and Cash Equivalents,67528.2,144162.5,182433.7,63448.0,35970.0,48830.0,72240.0,86460.0,68450.0,124880.0
4,Cash,18413.6,14678.6,18618.9,62920.6,35970.0,48830.0,72240.0,86460.0,68450.0,124880.0
5,Cash Equivalents,49114.6,129483.9,163814.8,527.4,0.0,0.0,0.0,0.0,0.0,0.0
6,Short Term Investments,9290.4,11584.7,14926.0,223591.5,421060.0,377980.0,347150.0,270790.0,315380.0,359450.0
7,"Available-for-Sale Securities,...",69.6,69.7,0.0,201719.5,219990.0,252170.0,247720.0,244220.0,241200.0,269090.0
8,Other Short Term Investments,3351.5,7278.7,0.0,4914.4,4700.0,28460.0,98800.0,26570.0,74180.0,90360.0
9,"Trading/ Listed Investments, C...",5869.3,4236.3,14926.0,16957.6,196370.0,97350.0,630.0,0.0,0.0,0.0
